In [38]:
from sympy import symbols
import sympy as sp
import numpy as np
from sympy.utilities.lambdify import lambdify, lambdastr
from matplotlib import pyplot as plt
from functools import cache

In [44]:
a_1 = 0.0134
b_1 = 1
c_1 = 4.35 * 10e-4
m_1 = 1
alpha_0 = 1.94 * 10e-2
delta = 1.5 * 10e3
gamma = 0.2 * 10e-2
l = 10
T_0 = 300
R = 0.5
F_0 = 50
X = sp.Symbol('x')
T = sp.Function('T')(X)
y_i_prev, y_i, y_i_next = symbols('y_im1, y_i, y_ip1')

In [45]:
alpha = alpha_0 * (y_i / delta - 1) ** 4 + gamma
p = 2 / R * alpha
f = 2 * T_0 / R * alpha
k = a_1 * (b_1 + c_1 * y_i ** m_1)
k_prev = k.subs(y_i, y_i_prev)
k_next = k.subs(y_i, y_i_next)

In [46]:
alpha, p, f, k

(0.194*(6.66666666666667e-5*y_i - 1)**4 + 0.02,
 0.776*(6.66666666666667e-5*y_i - 1)**4 + 0.08,
 232.8*(6.66666666666667e-5*y_i - 1)**4 + 24.0,
 5.829e-5*y_i + 0.0134)

In [47]:
k_prev

5.829e-5*y_im1 + 0.0134

In [48]:
n = 100
h = l / n
x = np.linspace(0, l, n+1, endpoint=True)

In [49]:
def eval_sp(func, i):
    func = func.subs(y_i, y[i]).subs(y_i_prev, y[i-1])
    if i < n:
        func = func.subs(y_i_next, y[i+1])
    return func

In [50]:
def get_a_i(i):
    if i == 0:
        return sp.Float(0)
    return (k_prev + k) / 2

def get_b_i(i):
    if i == 0:
        return (k + k_next) / 2
    if i == n:
        return (k_prev + k) / 2 + p * h ** 2 + alpha * h
    return (k_prev + k) / 2 + (k + k_next) / 2 + p * h ** 2

def get_c_i(i):
    if i == 0:
        return (k + k_next) / 2
    if i == n:
        return sp.Float(0)
    return (k + k_next) / 2

def get_d_i(i):
    if i == 0:
        return f * h ** 2 + F_0 * h
    if i == n:
        return f * h ** 2 + alpha * T_0 * h
    return f * h ** 2

In [51]:
y = np.full(n+2, T_0)
A_prime = np.zeros_like(x)
B_prime = np.zeros_like(x)
C_prime = np.zeros_like(x)
D_prime = np.zeros_like(x)
A_lambdas = [None for i in range(n+1)]
B_lambdas = [None for i in range(n+1)]
C_lambdas = [None for i in range(n+1)]
D_lambdas = [None for i in range(n+1)]

In [52]:
def my_lambdify(func):
    return lambdify([y_i_prev, y_i, y_i_next], func)

In [56]:
for i, x_i in enumerate(x):
    A_i = get_a_i(i)
    B_i = get_b_i(i)
    C_i = get_c_i(i)
    D_i = get_d_i(i)
    G_i = A_i * y_i_prev + B_i * y_i - C_i * y_i_next + D_i
    # print(lambdastr([y_i_prev, y_i, y_i_next], sp.diff(G_i, y_i_next)))
    A_lambdas[i] = my_lambdify(sp.diff(G_i, y_i_prev))
    B_lambdas[i] = my_lambdify(-sp.diff(G_i, y_i))
    C_lambdas[i] = my_lambdify(sp.diff(G_i, y_i_next))
    D_lambdas[i] = my_lambdify(G_i)

In [58]:
while True:
    for i, x_i in enumerate(x):
        A_prime[i] = A_lambdas[i](y[i-1], y[i], y[i+1])
        B_prime[i] = B_lambdas[i](y[i-1], y[i], y[i+1])
        C_prime[i] = C_lambdas[i](y[i-1], y[i], y[i+1])
        D_prime[i] = D_lambdas[i](y[i-1], y[i], y[i+1])

    eta = np.zeros_like(x)
    xi = np.zeros_like(x)
    delta_y = np.zeros_like(y)

    for i in range(1, n+1):
        xi[i] = C_prime[i-1] / (B_prime[i-1] - A_prime[i-1] * xi[i-1])
        eta[i] = (A_prime[i-1] * eta[-1] + D_prime[i-1]) / (B_prime[i-1] - A_prime[i-1] * xi[i-1])

    for i in reversed(range(1, n+1)):
        if i == n:
            delta_y[i] = (A_prime[n] * eta[n] + D_prime[n]) / (B_prime[n] - A_prime[n] * xi[n])
        else:
            delta_y[i] = xi[i+1] * delta_y[i+1] + eta[i+1]

    for i in range(1, n):
        y[i] = y[i-1] + delta_y[i]

    m = np.argmax(abs(delta_y / y))
    print(delta_y)
    break

[     0   -294   2378   -211    799    679   1074   1303   1577   1840
   2106   2372   2638   2905   3173   3441   3710   3980   4250   4522
   4794   5067   5342   5617   5893   6171   6450   6730   7012   7295
   7580   7866   8153   8442   8733   9026   9321   9618   9917  10218
  10521  10827  11135  11445  11758  12074  12393  12714  13039  13367
  13698  14033  14371  14713  15059  15409  15763  16122  16485  16853
  17226  17604  17987  18376  18771  19172  19579  19992  20413  20841
  21276  21719  22170  22629  23098  23576  24063  24561  25069  25589
  26120  26663  27219  27788  28372  28970  29584  30214  30861  31525
  32207  32906  33618  34331  35019  35616  35966  35707  34010  29003
 -13190      0]


In [ ]:

for i in range(1, n):
    y_final[i] = y_final[i-1] + delta_y[i]

In [ ]:
plt.plot(delta_y)

In [ ]:
delta_y

In [ ]:
F_0